In [ ]:
import matplotlib.pyplot as plt

def plot_values(data_dict):
    # 绘制数值变化曲线
    plt.figure(figsize=(10, 6))
    
    # 遍历字典中的每个键值对
    for label, values in data_dict.items():
        plt.plot(values, label=label)  # 绘制曲线并设置标签
    
    plt.xlabel('Timestep')
    plt.ylabel('Value')
    plt.title('Variation over Timesteps')
    plt.grid(True)
    plt.legend()  # 显示图例
    plt.show()

In [ ]:
import torch

beta_start = 0.0001
beta_end = 0.02
timesteps = 1000


betas = torch.cat([torch.zeros(1), torch.linspace(beta_start, beta_end, timesteps)])

alphas = 1.0 - betas
alphas_bar = torch.cumprod(alphas, dim=0)
sigmas = torch.sqrt(betas)

gamma = torch.tensor(5)
snr = alphas_bar / (1 - alphas_bar)
min_snr=torch.minimum(gamma/snr,torch.tensor(1))
# min_snr = torch.minimum(snr, gamma)

In [ ]:
# 找到第一个大于 gamma 的索引
first_index_over_gamma = (min_snr >=1).nonzero(as_tuple=True)[0][0]
print(first_index_over_gamma)

In [ ]:
data={
    "min_snr":min_snr,
    # "snr":snr,
}

plot_values(data)